# In this notebook, I will generate amino acid X --> amino acid Y sidechain mutations and generate protein maps thereof for the 19-by-19 (omitting proline) amino acid transforms.

import some libraries

In [1]:
from __future__ import absolute_import

from perses.dispersed import feptasks
from perses.utils.openeye import *
from perses.utils.data import load_smi
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from perses.rjmc.topology_proposal import TopologyProposal, TwoMoleculeSetProposalEngine, SystemGenerator,SmallMoleculeSetProposalEngine, PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
import simtk.openmm.app as app

from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState

import pymbar
import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import pickle
import mdtraj as md
from io import StringIO
from openmmtools.constants import kB
import logging
import os
import dask.distributed as distributed
import parmed as pm
from collections import namedtuple
from typing import List, Tuple, Union, NamedTuple
from collections import namedtuple
import random
#beta = 1.0/(kB*temperature)
import itertools
import os
from openeye import oechem
from perses.utils.smallmolecules import render_atom_mapping
from perses.tests.utils import validate_endstate_energies

ENERGY_THRESHOLD = 1e-6
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/perses-0.4.0-py3.6.egg/perses/rjmc/coordinate_numba.py:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  n_2 = np.dot(a, a)
/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/perses-0.4.0-py3.6.egg/perses/rjmc/coordinate_numba.py:65: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_ang = np.dot(angle_rotation_matrix, d_r)
/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/perses-0.4.0-py3.6.egg/perses/rjmc/coordinate_numba.py:71: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_torsion = np.dot(torsion_rotation_matrix, d_ang)
/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/perses-0.4.0-py3.6.egg/perses/rjmc/coordinate_numba.py:129: NumbaPerformanceWar

pull alanine dipeptide template for funsies, then make a system generator

In [2]:
import openmmtools.testsystems as ts
atp = ts.AlanineDipeptideVacuum(constraints = app.HBonds, hydrogenMass = 4 * unit.amus)
for chain in atp.topology.chains():
    print(chain.id)
    for res in chain.residues():
        print(res.name, res.id, res.index)
        for atom in res.atoms():
            print(atom.name, atom.index, atom.id)
        print()

forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
system_generator = SystemGenerator(forcefield_files,
                                   barostat = None,
                                   forcefield_kwargs = {'removeCMMotion': False, 
                                                        'ewaldErrorTolerance': 1e-4, 
                                                        'nonbondedMethod': app.NoCutoff,
                                                        'constraints' : app.HBonds, 
                                                        'hydrogenMass' : 4 * unit.amus})

atp.system = system_generator.build_system(atp.topology)



1
ACE 1 0
H1 0 1
CH3 1 2
H2 2 3
H3 3 4
C 4 5
O 5 6

ALA 2 1
N 6 7
H 7 8
CA 8 9
HA 9 10
CB 10 11
HB1 11 12
HB2 12 13
HB3 13 14
C 14 15
O 15 16

NME 3 2
N 16 17
H 17 18
C 18 19
H1 19 20
H2 20 21
H3 21 22



create a point mutation engine...we will omit the following code since it is just for troubleshooting's sake

In [ ]:
# point_mutation_engine = PointMutationEngine(wildtype_topology = atp.topology,
#                                             system_generator = system_generator,
#                                             chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
#                                             max_point_mutants = 1,
#                                             residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
#                                             allowed_mutations = [('2', 'ARG')], #the residue ids allowed to mutate with the three-letter code allowed to change
#                                             aggregate = True) #always allow aggregation

# geometry_engine = FFAllAngleGeometryEngine(metadata=None, 
#                                        use_sterics=False, 
#                                        n_bond_divisions=100, 
#                                        n_angle_divisions=180, 
#                                        n_torsion_divisions=360, 
#                                        verbose=True, 
#                                        storage=None, 
#                                        bond_softening_constant=1.0, 
#                                        angle_softening_constant=1.0, 
#                                        neglect_angles = False, 
#                                        use_14_nonbondeds = False)

# topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = atp.system,
#                               current_topology = atp.topology)

# new_positions, logp_proposal = geometry_engine.propose(topology_proposal, atp.positions, beta)

# #now to asn
# point_mutation_engine = PointMutationEngine(wildtype_topology = topology_proposal._new_topology,
#                                             system_generator = system_generator,
#                                             chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
#                                             max_point_mutants = 1,
#                                             residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
#                                             allowed_mutations = [('2', 'ASN')], #the residue ids allowed to mutate with the three-letter code allowed to change
#                                             aggregate = True) #always allow aggregation

# topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = topology_proposal._new_system,
#                               current_topology = topology_proposal._new_topology)
# new_positions, logp_proposal = geometry_engine.propose(topology_proposal, new_positions, beta)
# render_atom_mapping('test.png', old_oemol_sidechain, new_oemol_sidechain, local_map_stereo_sidechain)

In [ ]:
# for chain in topology_proposal.old_topology.chains():
#     print(chain.id)
#     for res in chain.residues():
#         print(res.name, res.id, res.index)
#         for atom in res.atoms():
#             print(atom.name, atom.index)
#         print()

In [ ]:
# for chain in topology_proposal.new_topology.chains():
#     print(chain.id)
#     for res in chain.residues():
#         print(res.name, res.id, res.index)
#         for atom in res.atoms():
#             print(atom.name, atom.index)
#         print()

render the atom mapping

In [ ]:
# [atom.GetIdx() for atom in old_oemol_sidechain.GetAtoms()]
# for atom in old_oemol_sidechain.GetAtoms():
#     print(atom.GetIdx())
    

In [ ]:
# mol1 = oechem.OEGraphMol(old_oemol_sidechain)
# [atom.GetIdx() for atom in mol1.GetAtoms()]

In [ ]:
# render_atom_mapping('test.png', old_oemol_sidechain, new_oemol_sidechain, local_map_stereo_sidechain)

In [ ]:
# point_mutation_engine = PointMutationEngine(wildtype_topology = atp.topology,
#                                             system_generator = system_generator,
#                                             chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
#                                             max_point_mutants = 1,
#                                             residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
#                                             allowed_mutations = [('2', 'ASN')], #the residue ids allowed to mutate with the three-letter code allowed to change
#                                             aggregate = True) #always allow aggregation

# topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = atp.system,
#                               current_topology = atp.topology)

## Now make a few functions to automate the mapping pipeline 

In [3]:
def create_new_pdb(topology, positions, output_pdb = 'test_new.pdb'):
    """
    create a pdb of the geometry proposal (only new system)
    """
    import mdtraj as md
    _positions =  np.array(positions.value_in_unit(unit.nanometer))
    print(_positions)
    traj = md.Trajectory(_positions, md.Topology.from_openmm(topology))
    traj.save(output_pdb)

In [4]:
def validate_geometry(topology_proposal, current_positions, beta):
    """generate a geometry proposal"""
    validated = True
    geometry_engine = FFAllAngleGeometryEngine(metadata=None, 
                                           use_sterics=False, 
                                           n_bond_divisions=100, 
                                           n_angle_divisions=180, 
                                           n_torsion_divisions=360, 
                                           verbose=True, 
                                           storage=None, 
                                           bond_softening_constant=1.0, 
                                           angle_softening_constant=1.0, 
                                           neglect_angles = False, 
                                           use_14_nonbondeds = False)
    try:
        new_positions, logp_proposal = geometry_engine.propose(topology_proposal, current_positions, beta)
    except Exception as e:
        print(f"geometry failure {e}")
        new_positions, logp_proposal = None, None
        validated = False
    
    return validated, new_positions
    

In [5]:
def validate_htf(topology_proposal, old_positions, new_positions):
    """this isn't absolutely critical to make sure that the hybrid topology factory function will work"""
    validated = True
    try:
        htf = HybridTopologyFactory(topology_proposal = topology_proposal,
                     current_positions = old_positions,
                     new_positions = new_positions,
                     use_dispersion_correction = False,
                     functions=None,
                     softcore_alpha = None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new = False,
                     neglected_new_angle_terms = [],
                     neglected_old_angle_terms = [],
                     softcore_LJ_v2 = True,
                     softcore_electrostatics = True,
                     softcore_LJ_v2_alpha = 0.85,
                     softcore_electrostatics_alpha = 0.3,
                     softcore_sigma_Q = 1.0,
                     interpolate_old_and_new_14s = False,
                     omitted_terms = None)
    except Exception as e:
        print(f"htf failure: {e}")
        htf = None
        validated = False
    
    return validated, htf

In [6]:
def run_pipeline(old_topology, old_system, old_positions, system_generator, mutations_from = 'ALA'):
    """
    run the pipeline to generate a protein mutation from all of the dipeptides
    """
    allowed_mutations = [item for item in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU','GLY','HIS','ILE','LEU','LYS', 'MET', 'PHE', 'SER', 'THR', 'TRP', 'TYR','VAL'] if item != mutations_from]
    print(f"allowed_mutations: {allowed_mutations}")
    results = {}
    for mutation in allowed_mutations:
        print(mutation)
        
        point_mutation_engine = PointMutationEngine(wildtype_topology = old_topology,
                                            system_generator = system_generator,
                                            chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
                                            max_point_mutants = 1,
                                            residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
                                            allowed_mutations = [('2', mutation)], #the residue ids allowed to mutate with the three-letter code allowed to change
                                            aggregate = True) #always allow aggregation
        
        topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = old_system,
                              current_topology = old_topology)
        
        validated_geometry, new_positions = validate_geometry(topology_proposal, old_positions, beta)
        try:
            _validated_htf, htf = validate_htf(topology_proposal, old_positions, new_positions)
            render_atom_mapping(f"{mutations_from}_to_{mutation}.png", old_oemol_sidechain, new_oemol_sidechain, local_map_stereo_sidechain)
            results[(mutations_from, mutation)] = (topology_proposal, htf)

        except Exception as e:
            print(f"{mutations_from} to {mutation}; {e}")
    
    return results
        
                
            
        
        
        
        
    
    
    
    

### all transforms

note: we first have to make a topology proposal from ALA to all of the other amino acids in order to generate a new topology for ACE-X-NME, from which we can transform X --> Y

In [7]:
from_alanine_results = run_pipeline(atp.topology, atp.system, atp.positions, system_generator, 'ALA')
_rest_to_remove = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU','GLY','HIS','ILE','LEU','LYS', 'MET', 'PHE', 'SER', 'THR', 'TRP', 'TYR','VAL']
_all_results = [from_alanine_results]
for _next_mutation in _rest_to_remove[1:]:
    print(f"starting from: {_next_mutation}")
    topology, system, positions = from_alanine_results[('ALA', _next_mutation)][0]._new_topology, from_alanine_results[('ALA', _next_mutation)][0]._new_system, from_alanine_results[('ALA', _next_mutation)][1]._new_positions
    _all_results.append(run_pipeline(topology, system, positions, system_generator, _next_mutation))

allowed_mutations: ['ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
ARG
ASN
ASP
CYS
GLN
GLU
GLY
HIS
ILE
LEU
LYS
MET
PHE
SER
THR
TRP
TYR
VAL
starting from: ARG
allowed_mutations: ['ALA', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
ALA


ASN


ASP
CYS
GLN
GLU


GLY
HIS
ILE
LEU
LYS


MET
PHE
SER
THR


...

PHE
SER


THR
TRP
VAL
starting from: VAL
allowed_mutations: ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'SER', 'THR', 'TRP', 'TYR']
ALA
ARG
ASN
ASP
CYS
GLN
GLU
GLY
HIS


ILE
LEU
LYS
MET
PHE


SER
THR
TRP


TYR


In [8]:
#let's group all of these into a single file
os.system(f"mkdir sidechain_maps")

0

In [9]:
os.system(f"mv *.png sidechain_maps")

0

CA warnings in the geometry proposal are indeed a consequence of the following:
    when the CB does not map, but all of the other backbones do, and the CA is chiral (which is in every case with the exception of GLY), then 3 valencies of 4 are filled, so a chiral improper torsion can place the CB into the wrong       orientation, which could cause a steric clash.  if 3/4 valencies are occupied on a core or unique new atom, then a bias torsion will not be used to place the atoms.


i also think that the topology proposal warnings: "WARNING:proposal_generator:there may be a geometry problem!  It is advisable to conduct a manual atom map." are a consequence of the class running into an error with chiral atom placements/exchanges (the method is not exactly robust, as it only checks the neighbors of atoms to retain chirality).  on the other hand, if the transformation fails to find any atom maps, it will return an empty dict.  in the event of protein changes, this is not strictly terrible since the backbone still maps, so the simulation should still be able to run

a note on limitations: HIS is problematic, since there are two tautomers; also, titratable residues have single atom maps, which are always in their physiologically dominant forms.  charge perturbations are also not yet supported

## Now that we can make a mapping that won't fail, we should try to see if we can make a simple repex simulation run!!!

go to _protein_test_mutation.ipynb_